In [167]:
from datetime import datetime
import requests

response = requests.get('http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophies&a=htab&ob=ra,seq&c=o&o=51%20peg')

print(response.status_code)

200


In [168]:
import bs4

# Obtener el html

soup = bs4.BeautifulSoup(response.text, 'html.parser')

xpath_title = '//table[@class="datatable"]/tbody/tr/td[7]/text()'

query = soup.select('table[class="datatable"] tbody tr')

print(len(query))
#print(query[1].select('td')[6])
dates = []
for i in range(0, len(query)):
    #print(query[i].select('td')[6].text)
    dates.append(query[i].select('td')[6].text)
print('dates: ', len(dates))

183
dates:  183


In [169]:
import bs4

# Obtener el html

soup = bs4.BeautifulSoup(response.text, 'html.parser')

xpath_title = '//table[@class="datatable"]/thead/tr/th/a/text()'


query = soup.select('table[class="datatable"] thead th')


In [170]:
print(len(query))

header = []
for i in range(0,len(query)):
    header.append(query[i].text)

lista = ['date', 'fiber_b', 'sn26', 'view_spec', 'search_ccf']    
posicion = []
for i in range(0,len(header)):
    for j in range(0,len(lista)):
        if header[i] == lista[j]:
            posicion.append(i)

diccionario ={}
for i in range(0, len(lista)):
    diccionario[lista[i]]=posicion[i]
diccionario

17


{'date': 6, 'fiber_b': 8, 'sn26': 10, 'view_spec': 11, 'search_ccf': 16}

In [171]:
query = soup.select('table[class="datatable"] tbody tr')

#for i in range(0,len(query)):
#    print(query[i]['href'])
print(query[3].select('td')[16].select('a'))

[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =825680]">search_CCF</a>]


In [172]:
def article_links(self):
    link_list = []
    for link in self._select(self._queries['homepage_article_links']):
        if link and link.has_attr('href'):
            link_list.append(link)

    return set(link['href'] for link in link_list)

In [215]:
def column():    
    query = soup.select('table[class="datatable"] tbody tr')
    
    select_a = []
    for i in range(0, len(query)):
        data = query[i].select('td')[16].select('a')
        select_a.append(data)
    link_list = []
    none_url = []
    for index, link in enumerate(select_a):
        if len(link) ==0:
            link = [None]
            none_url.append(index)
        if link[0] and link[0].has_attr('href'):
            link_list.append(link)

            
    return list(set(link[0]['href'] for link in link_list)), none_url


In [216]:
a, b = column()
print(a[0])
print(len(b))

sophie.cgi?n=sophiecc&a=hexp&fql=[seq =1003430]
7


In [144]:
#print(column())
abc = column()

resultados = []
for i in range(0,len(abc)):
    #print(abc[i])
    if len(abc[i]) ==0:
        abc[i] = None
    print(abc[i])
        
    

[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =671480]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =815760]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =815780]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =825680]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =847890]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =848960]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =1014070]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =1203340]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =1217920]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =1225370]">search_CCF</a>]
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =1244250]">search_CCF</a>]
None
None
None
None
None
None
None
[<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =1

# Borrar amp; y corregir los espacios

In [ ]:
href="sophie.cgi?n=sophiecc&a=hexp&fql=[seq =671480]"


In [202]:
set([1,2,4,2,1])

{1, 2, 4}